In [53]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0  attrdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import string
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging


In [3]:
batch_size = 16
max_len = 256

In [4]:
df_train = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/IMDb/IMDb_train.tsv", sep="\t", header=None)
df_train = df_train.iloc[:, 0:2]
df_train.columns = ["text", "label"]
print(df_train.shape)
df_train.head()

(25000, 2)


,text,label
0,The Unborn is a pretty good low-budget horror ...,1
1,Vincente Minnelli directed some of the most ce...,1
2,"The first time I saw this, I didn't laugh too ...",1
3,This is a great movie for all Generation X'ers...,1
4,I first saw this absolutely riveting documenta...,1


In [5]:
df_test = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/IMDb/IMDb_test.tsv", sep="\t", header=None)
df_test = df_test.iloc[:, 0:2]
df_test.columns = ["text", "label"]
print(df_test.shape)
df_test.head()

(25000, 2)


,text,label
0,"Susan Sarandon is, for lack of a better word, ...",1
1,Seeing Laurel without Hardy in a film seems st...,1
2,I was recently at a sleepover birthday party w...,1
3,This movie took me by surprise. The opening cr...,1
4,A widely unknown strange little western with m...,1


In [6]:
df_train["label"].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [7]:
df_train["text"].map(len).median()

979.0

In [8]:
df_train["text"].map(len)

0         639
1        4018
2         921
3        1582
4        1526
         ... 
24995     472
24996     751
24997     964
24998    6041
24999    1405
Name: text, Length: 25000, dtype: int64

# 前処理

In [9]:
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


In [10]:
df_train["text"].map(preprocessing_text)
preprocessing_text(df_train["text"][0])

'The Unborn is a pretty good low budget horror movie exploiting the fears associated with pregnancy .  It s very well acted by the always good Brooke Adams and b movie stalwart James Karen ,  although the supporting cast is pretty average for a b grader .  The music ,  by Gary Numan of all people ,  is good too .  Henry Dominic s script is quite intelligent for this sort of thing ,  although there is a hint of misogyny about it .  Rodman Fender s direction is merely adequate ,  and there are some unnecessary cheap scares .  If you re a fan of Adams ,  whose movie career is nowhere near as illustrious as it should be ,  check it out  she s great ,  as always . '

# データセット作成

In [11]:
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    # self.uid = uid
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.y)

  def encode(self, tokenizer, text):

      # 前処理
      text = preprocessing_text(text)
      

      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    label = self.y[index]
    # userID = self.uid[index]
    ids = []
    mask = []
    inputs = self.encode(tokenizer=self.tokenizer, text=text)
    ids.append(torch.LongTensor(inputs['input_ids']))
    mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      'label': label,
      'text':text,
      # 'userID':userID
    }

In [12]:
# tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [13]:
X = df_train["text"].values
y = df_train["label"].values

In [14]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, train_size=0.75)

print(len(X_train))
print(len(y_train))
print(len(X_eval))
print(len(y_eval))

X_test = df_test["text"].values
y_test = df_test["label"].values
print(len(X_test))
print(len(y_test))

18750
18750
6250
6250
25000
25000


In [15]:
dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len=max_len)
dataset_eval = CreateDataset(X_eval, y_eval, tokenizer, max_len=max_len)
dataset_test = CreateDataset(X_test, y_test, tokenizer, max_len=max_len)

print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

18750
6250
25000


In [32]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=False, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, pin_memory=True)

# Tensorの挙動

In [57]:
config = {'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

from attrdict import AttrDict
config = AttrDict(config)
config.hidden_size



768

In [60]:
# BERT用にLayerNormalization層を定義します。
# 実装の細かな点をTensorFlowに合わせています。


class BertLayerNorm(nn.Module):
    """LayerNormalization層 """

    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size))  # weightのこと
        self.beta = nn.Parameter(torch.zeros(hidden_size))  # biasのこと
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta


In [61]:
# BERTのEmbeddingsモジュールです
from torch import nn

class BertEmbeddings(nn.Module):
    """文章の単語ID列と、1文目か2文目かの情報を、埋め込みベクトルに変換する
    """

    def __init__(self, config):
        super(BertEmbeddings, self).__init__()

        # 3つのベクトル表現の埋め込み

        # Token Embedding：単語IDを単語ベクトルに変換、
        # vocab_size = 30522でBERTの学習済みモデルで使用したボキャブラリーの量
        # hidden_size = 768 で特徴量ベクトルの長さは768
        self.word_embeddings = nn.Embedding(
            config.vocab_size, config.hidden_size, padding_idx=0)
        # （注釈）padding_idx=0はidx=0の単語のベクトルは0にする。BERTのボキャブラリーのidx=0が[PAD]である。

        # Transformer Positional Embedding：位置情報テンソルをベクトルに変換
        # Transformerの場合はsin、cosからなる固定値だったが、BERTは学習させる
        # max_position_embeddings = 512　で文の長さは512単語
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size)

        # Sentence Embedding：文章の1文目、2文目の情報をベクトルに変換
        # type_vocab_size = 2
        self.token_type_embeddings = nn.Embedding(
            config.type_vocab_size, config.hidden_size)

        # 作成したLayerNormalization層
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        # Dropout　'hidden_dropout_prob': 0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, token_type_ids=None):
        '''
        input_ids： [batch_size, seq_len]の文章の単語IDの羅列
        token_type_ids：[batch_size, seq_len]の各単語が1文目なのか、2文目なのかを示すid
        '''

        # 1. Token Embeddings
        # 単語IDを単語ベクトルに変換
        words_embeddings = self.word_embeddings(input_ids)

        # 2. Sentence Embedding
        # token_type_idsがない場合は文章の全単語を1文目として、0にする
        # そこで、input_idsと同じサイズのゼロテンソルを作成
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        # 3. Transformer Positional Embedding：
        # [0, 1, 2 ・・・]と文章の長さだけ、数字が1つずつ昇順に入った
        # [batch_size, seq_len]のテンソルposition_idsを作成
        # position_idsを入力して、position_embeddings層から768次元のテンソルを取り出す
        seq_length = input_ids.size(1)  # 文章の長さ
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        # 3つの埋め込みテンソルを足し合わせる [batch_size, seq_len, hidden_size]
        embeddings = words_embeddings + position_embeddings + token_type_embeddings

        # LayerNormalizationとDropoutを実行
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings


In [72]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [74]:
batch = next(iter(dataloader_train))
# input_ids = batch["ids"][0].to(device)
print(input_ids.size())

torch.Size([16, 256])


In [75]:
be = BertEmbeddings(config)

In [76]:
seq_length = input_ids.size(1)
position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
print(position_ids)
position_embeddings = be.position_embeddings(position_ids)
position_embeddings

tensor([[  0,   1,   2,  ..., 253, 254, 255],
        [  0,   1,   2,  ..., 253, 254, 255],
        [  0,   1,   2,  ..., 253, 254, 255],
        ...,
        [  0,   1,   2,  ..., 253, 254, 255],
        [  0,   1,   2,  ..., 253, 254, 255],
        [  0,   1,   2,  ..., 253, 254, 255]], device='cuda:0')


RuntimeError: ignored